<a href="https://colab.research.google.com/github/zerualem/pi-project/blob/master/Pi_data_using_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
!dir

My\ Drive


In [3]:
%cd My\ Drive/PIPM_Data_R_code/Analysis

/gdrive/My Drive/PIPM_Data_R_code/Analysis


## Load the data from drive

In [4]:
import pandas as pd

all_dmy = pd.read_csv('all_piData_dmy.csv')
all_dmy.shape

(26258, 238)

In [0]:
all_dmy.columns[all_dmy.columns.str.startswith('pressure_injury_poa')]

Index(['pressure_injury_poa.Yes'], dtype='object')

In [6]:
y = all_dmy[['pi']]
X = all_dmy.drop(['pi', 'pi_dayfromadmit', 'pressure_injury_poa.Yes'], axis=1)
X.shape

(26258, 235)

## Prepare the data

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [0]:
print(X_train.shape)
print(y_train.shape)

(19693, 235)
(19693, 1)


### Scale the data

Scale the data using min-max scaler from Scikit-learn package.

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print(scaler.fit(X_train))
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

MinMaxScaler(copy=True, feature_range=(0, 1))


## Build neural network using keras from Tensorflow

In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.14.0


In [0]:
#y_train = keras.utils.to_categorical(y_train)
#y_test = keras.utils.to_categorical(y_test)
#y_train.shape

(19693, 2)

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(X_train.shape[1],)),
    keras.layers.Dense(256, activation=tf.nn.relu),
    #keras.layers.Dropout(rate=0.3),
    keras.layers.Dense(256, activation=tf.nn.relu),
    #keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

Some references


*   https://www.tensorflow.org/beta/tutorials/keras/imbalanced_data


*   https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc 




In [0]:
metrics = [
      keras.metrics.Accuracy(name='accuracy'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc')]

In [0]:
adam = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=metrics)

In [70]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 235)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 256)               60416     
_________________________________________________________________
dense_29 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_30 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 129       
Total params: 159,233
Trainable params: 159,233
Non-trainable params: 0
_________________________________________________________________


In [81]:
EPOCHS = 30
BATCH_SIZE = 1024

history = model.fit( 
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS)

Epoch 1/30
19693/19693 [==============================] - 1s 29us/sample - loss: 0.8524 - accuracy: 0.7626 - tp: 411.0000 - fp: 479.0000 - tn: 18230.0000 - fn: 573.0000 - precision: 0.4618 - recall: 0.4177 - auc: 0.7458
Epoch 2/30
19693/19693 [==============================] - 1s 29us/sample - loss: 0.8562 - accuracy: 0.7712 - tp: 411.0000 - fp: 451.0000 - tn: 18258.0000 - fn: 573.0000 - precision: 0.4768 - recall: 0.4177 - auc: 0.7441
Epoch 3/30
19693/19693 [==============================] - 1s 29us/sample - loss: 0.8185 - accuracy: 0.7398 - tp: 442.0000 - fp: 507.0000 - tn: 18202.0000 - fn: 542.0000 - precision: 0.4658 - recall: 0.4492 - auc: 0.7636
Epoch 4/30
19693/19693 [==============================] - 1s 29us/sample - loss: 0.8152 - accuracy: 0.7337 - tp: 421.0000 - fp: 485.0000 - tn: 18224.0000 - fn: 563.0000 - precision: 0.4647 - recall: 0.4278 - auc: 0.7538
Epoch 5/30
19693/19693 [==============================] - 1s 29us/sample - loss: 0.7832 - accuracy: 0.7347 - tp: 429.000

### Evaluate the model on the test set

In [82]:
results = model.evaluate(X_test, y_test)

for name, value in zip(model.metrics_names, results):
  print(name, ': ', value)

6565/6565 [==============================] - 0s 54us/sample - loss: 0.6894 - accuracy: 0.5593 - tp: 135.0000 - fp: 177.0000 - tn: 6042.0000 - fn: 211.0000 - precision: 0.4327 - recall: 0.3902 - auc: 0.7504
loss :  0.6893596469169461
accuracy :  0.5593298
tp :  135.0
fp :  177.0
tn :  6042.0
fn :  211.0
precision :  0.43269232
recall :  0.3901734
auc :  0.7503911


In [83]:
from sklearn import metrics
predicted = model.predict(X_test)
print(metrics.classification_report(y_test.values, (predicted.astype(np.int32))))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6219
           1       0.60      0.15      0.24       346

    accuracy                           0.95      6565
   macro avg       0.78      0.57      0.61      6565
weighted avg       0.94      0.95      0.94      6565



### Using class weights to tackle data inbalance

In [0]:
neg, pos = np.bincount(y_train.values.flatten())
weight_for_0 =  1 #1 / neg
weight_for_1 = 1  #1 / pos

class_weight = {0: weight_for_0, 1: weight_for_1}

In [0]:
class_weight

{0: 5.345021112833396e-05, 1: 0.0010162601626016261}

In [90]:
EPOCHS = 20
BATCH_SIZE = 2048

weighted_model = model

weighted_history = weighted_model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weight)

Epoch 1/20
19693/19693 [==============================] - 1s 27us/sample - loss: 0.5455 - accuracy: 0.4514 - tp: 450.0000 - fp: 532.0000 - tn: 18177.0000 - fn: 534.0000 - precision: 0.4582 - recall: 0.4573 - auc: 0.7918
Epoch 2/20
19693/19693 [==============================] - 1s 27us/sample - loss: 0.5374 - accuracy: 0.4796 - tp: 426.0000 - fp: 471.0000 - tn: 18238.0000 - fn: 558.0000 - precision: 0.4749 - recall: 0.4329 - auc: 0.7916
Epoch 3/20
19693/19693 [==============================] - 1s 27us/sample - loss: 0.5269 - accuracy: 0.4536 - tp: 444.0000 - fp: 472.0000 - tn: 18237.0000 - fn: 540.0000 - precision: 0.4847 - recall: 0.4512 - auc: 0.7928
Epoch 4/20
19693/19693 [==============================] - 1s 27us/sample - loss: 0.5262 - accuracy: 0.4214 - tp: 444.0000 - fp: 515.0000 - tn: 18194.0000 - fn: 540.0000 - precision: 0.4630 - recall: 0.4512 - auc: 0.8030
Epoch 5/20
19693/19693 [==============================] - 1s 26us/sample - loss: 0.5216 - accuracy: 0.4374 - tp: 438.000

In [91]:
weighted_results = weighted_model.evaluate(X_test, y_test)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)

6565/6565 [==============================] - 1s 90us/sample - loss: 0.5957 - accuracy: 0.1956 - tp: 139.0000 - fp: 239.0000 - tn: 5980.0000 - fn: 207.0000 - precision: 0.3677 - recall: 0.4017 - auc: 0.7784
loss :  0.5957498071838334
accuracy :  0.19558263
tp :  139.0
fp :  239.0
tn :  5980.0
fn :  207.0
precision :  0.36772487
recall :  0.4017341
auc :  0.7783975


In [105]:
predicted = weighted_model.predict(X_test)
print(metrics.classification_report(y_test.values, (predicted.astype(np.int32))))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6219
           1       0.62      0.09      0.16       346

    accuracy                           0.95      6565
   macro avg       0.78      0.54      0.57      6565
weighted avg       0.93      0.95      0.93      6565



### Oversampling the minority class

In [103]:
# with default args this will oversample the minority class to have an equal
# number of observations
from imblearn.over_sampling import SMOTE
smote = SMOTE( sampling_strategy=0.6)
res_features, res_labels = smote.fit_sample(X_train, y_train.values.reshape(-1))

res_neg, res_pos = np.bincount(res_labels)
res_total = res_neg + res_pos
print('{} positive samples out of {} training samples ({:.2f}% of total)'.format(
    res_pos, res_total, 100 * res_pos / res_total))

11225 positive samples out of 29934 training samples (37.50% of total)


In [104]:
resampled_model = model
res_labels = np.reshape(res_labels,(-1,1))
resampled_history = resampled_model.fit(
    res_features, res_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weight)

Epoch 1/20
29934/29934 [==============================] - 1s 36us/sample - loss: 0.5756 - accuracy: 0.0614 - tp: 8743.0000 - fp: 2008.0000 - tn: 16701.0000 - fn: 2482.0000 - precision: 0.8132 - recall: 0.7789 - auc: 0.9101
Epoch 2/20
29934/29934 [==============================] - 1s 24us/sample - loss: 0.5645 - accuracy: 0.0617 - tp: 8825.0000 - fp: 2069.0000 - tn: 16640.0000 - fn: 2400.0000 - precision: 0.8101 - recall: 0.7862 - auc: 0.9120
Epoch 3/20
29934/29934 [==============================] - 1s 24us/sample - loss: 0.5902 - accuracy: 0.0609 - tp: 8731.0000 - fp: 2148.0000 - tn: 16561.0000 - fn: 2494.0000 - precision: 0.8026 - recall: 0.7778 - auc: 0.9062
Epoch 4/20
29934/29934 [==============================] - 1s 24us/sample - loss: 0.6728 - accuracy: 0.0592 - tp: 8426.0000 - fp: 2286.0000 - tn: 16423.0000 - fn: 2799.0000 - precision: 0.7866 - recall: 0.7506 - auc: 0.8888
Epoch 5/20
29934/29934 [==============================] - 1s 24us/sample - loss: 0.5726 - accuracy: 0.0612 -

In [102]:
resampled_results = resampled_model.evaluate(X_test, y_test)
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)

6565/6565 [==============================] - 1s 91us/sample - loss: 0.5431 - accuracy: 0.0062 - tp: 204.0000 - fp: 566.0000 - tn: 5653.0000 - fn: 142.0000 - precision: 0.2649 - recall: 0.5896 - auc: 0.8372
loss :  0.5431165092946041
accuracy :  0.00624524
tp :  204.0
fp :  566.0
tn :  5653.0
fn :  142.0
precision :  0.26493508
recall :  0.5895954
auc :  0.83717173


In [107]:
predicted = resampled_model.predict(X_test)
print(metrics.classification_report(y_test.values, (predicted.astype(np.int32))))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6219
           1       0.62      0.09      0.16       346

    accuracy                           0.95      6565
   macro avg       0.78      0.54      0.57      6565
weighted avg       0.93      0.95      0.93      6565

